<a href="https://colab.research.google.com/github/ConorD28/Baseball/blob/main/Get_IDs_Teams_Played_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybaseball
!pip install xlrd

from pybaseball import statcast, statcast_batter, playerid_lookup
import pandas as pd

# **Get Player IDs/Teams Played on in a Certain Year**

In [ ]:
data_year = statcast("2022-04-07", "2022-10-05") #"2021-04-01", "2021-10-03", "2023-03-30", "2023-10-01"

In [ ]:
df = pd.read_csv('2022 400 PA Hitter names.csv')

In [ ]:
def reformat_name(name):
    # Convert "Last, First" → "First Last"
    try:
        last, first = name.strip().split(", ")
        return f"{first} {last}"
    except:
        return None

def get_player_id(first_last_name):
    if not first_last_name:
        return None
    try:
        first, last = first_last_name.split(" ")
        results = playerid_lookup(last, first)
        if not results.empty:
            return results.iloc[0]['key_mlbam']
    except:
        return None
    return None

def add_ID_team_column(df, data):
    # Step 1: Reformat names and get MLBAM IDs
    df['name_fixed'] = df['Name'].apply(reformat_name)

    # #remove Jr. and II
    df['name_fixed'] = df['name_fixed'].str.replace(' Jr.', '', regex=False)
    df['name_fixed'] = df['name_fixed'].str.replace(' II', '', regex=False)

    df['mlbam_id'] = df['name_fixed'].apply(get_player_id)
    return df

    data = data[['batter', 'home_team', 'away_team', 'inning_topbot']].dropna(subset=['batter'])

    # Step 3: Create ID → team lookup
    id_to_team = {}
    unique_ids = df['mlbam_id'].dropna().unique()

    for pid in unique_ids:
        batter_data = data[data['batter'] == pid]
        if batter_data.empty:
            id_to_team[pid] = "Did not bat in this year"
            continue
        # Determine team from inning_topbot
        batter_data['batting_team'] = batter_data.apply(
            lambda row: row['away_team'] if row['inning_topbot'] == 'Top' else row['home_team'],
            axis=1
        )
        most_common_team = batter_data['batting_team'].mode().iloc[0]
        id_to_team[pid] = most_common_team

    # Step 4: Map teams back to original dataframe
    df['team'] = df['mlbam_id'].map(id_to_team)
    df['mlbam_id'] = df['mlbam_id'].fillna(0).astype(int)

    # Optional: drop helper columns
    df.drop(columns=['name_fixed'], inplace=True)

    return df

players2 = add_ID_team_column(df, data_year)
players2['mlbam_id_savant_search'] = '&batters_lookup%5B%5D=' + players2.mlbam_id.astype(str)

In [ ]:
#players2[players2['mlbam_id'].isnull()]

In [ ]:
players2.to_csv('2022 Players with IDs.csv') #manually get both values for rows with blanks